<a href="https://colab.research.google.com/github/savancs/product_recommender/blob/tori_branch/DistilBERT_DLAIEF23_PR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup - Check GPU!

## Packages/Libraries

In [1]:
# Step 1: Install the Hugging Face Transformers library
!pip install transformers
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [2]:
# Step 2: Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import wandb

from torch.utils.data import DataLoader, TensorDataset

In [3]:
!wandb login --relogin 59add063e6bbf968685833105867f8be02a2191a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Data

In [4]:
#product data

sephora_data = 'https://raw.githubusercontent.com/torrileigh/sephora_cleaned_11.26.23/main/cosmetic_p11.26.23.csv'
sephora_df = pd.read_csv(sephora_data)

sephora_df.head()

,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,ALGENIST,GENIUS Ultimate Anti-Aging Cream,$112.00,4.3,"Water, Caprylic/Capric Triglyceride, Hydrogent...",1,1,1,1,1
1,Moisturizer,ALGENIST,Overnight Restorative Cream,$94.00,4.4,"Water, Isopropyl Isostearate, Butyrospermum Pa...",0,0,0,0,0
2,Moisturizer,ALGENIST,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,$28.00,4.4,"Cyclopentasiloxane, Water, Alcohol Denat., Gly...",0,0,0,0,0
3,Moisturizer,ALGENIST,POWER Recharging Night Pressed Serum,$95.00,4.4,"Cocos Nucifera (Coconut) Water, Water (Aqua, E...",1,1,1,1,1
4,Moisturizer,ALGENIST,Firming & Lifting Neck Cream,$98.00,3.9,"Water, Isononyl Isononanoate, Dimethicone, But...",0,0,0,0,0


In [5]:
#we aren't using this now, may need later...

test_val_data = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/GPT%20Dataset%20PR.csv'
test_val_df = pd.read_csv(test_val_data)

test_val_df.head()

,Input Text,Product Type,Min Price,Max Price,Skin Concerns,Preferred Ingredients,Excluded Ingredients
0,Looking for a sun protect for dry skin $36-$70...,Sun Protect,36.0,70.0,Dry,Peptides,NaN
1,Looking for a eye cream for combination skin $...,Eye Cream,18.0,40.0,Combination,NaN,Phthalates
2,Looking for a moisturizer for dry skin $40-$72...,Moisturizer,40.0,72.0,Dry,NaN,Formaldehyde
3,Looking for a treatment for dry skin $20-$40 w...,Treatment,20.0,40.0,Dry,NaN,Parabens
4,Looking for a face mask for acne-prone skin $3...,Face Mask,37.0,44.0,Acne,Aloe Vera,NaN


In [6]:
import json
import requests

#now reads in full dataset
url = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/annotations-test%20combo.json'

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    annotated_df = json.loads(response.content)
else:
    print("Failed to download the data")


In [7]:
print(annotated_df.keys())

dict_keys(['classes', 'annotations'])


In [8]:
ner_annotations = annotated_df['annotations']
ner_classes = annotated_df['classes']

# checking it loaded well
for i, entry in enumerate(ner_annotations[:10]):
    if entry is not None:
        text, annotation_details = entry
        print(f"Entry {i+1}:")
        print("Text:", text)
        print("Annotations:", annotation_details)
        print("\n")
    else:
        print(f"Entry {i+1}: None\n")


Entry 1: None

Entry 2:
Text: Combination skin eye cream more than $6 but less than $16
Annotations: {'entities': [[0, 11, 'B-SKIN_CONCERN'], [12, 16, 'O'], [17, 20, 'B-PRODUCT_TYPE'], [21, 26, 'I-PRODUCT_TYPE'], [27, 31, 'O'], [32, 36, 'O'], [37, 39, 'B-MIN_PRICE'], [40, 43, 'O'], [44, 48, 'O'], [49, 53, 'O'], [54, 57, 'B-MAX_PRICE']]}


Entry 3:
Text: Cleanser with seaweed more than $41 but less than $95
Annotations: {'entities': [[0, 8, 'B-PRODUCT_TYPE'], [9, 13, 'O'], [14, 21, 'B-PREF_INGREDIENT'], [22, 26, 'O'], [27, 31, 'O'], [32, 35, 'B-MIN_PRICE'], [36, 39, 'O'], [40, 44, 'O'], [45, 49, 'O'], [50, 53, 'B-MAX_PRICE']]}


Entry 4:
Text: What's a good eye cream to use for skin that's sensitive?
Annotations: {'entities': [[0, 4, 'O'], [4, 6, 'O'], [7, 8, 'O'], [9, 13, 'O'], [14, 17, 'B-PRODUCT_TYPE'], [18, 23, 'I-PRODUCT_TYPE'], [24, 26, 'O'], [27, 30, 'O'], [31, 34, 'O'], [35, 39, 'O'], [40, 46, 'O'], [47, 56, 'B-SKIN_CONCERN'], [56, 57, 'O']]}


Entry 5:
Text: I need a face mask 

In [9]:
ner_annotations = annotated_df['annotations']
ner_classes = annotated_df['classes']

start_index = -10 if len(ner_annotations) >= 10 else -len(ner_annotations)

# Print the last 10 entries
for i, entry in enumerate(ner_annotations[start_index:], start=start_index + len(ner_annotations) + 1):
    if entry is not None:
        text, annotation_details = entry
        print(f"Entry {i}:")
        print("Text:", text)
        print("Annotations:", annotation_details)
        print("\n")
    else:
        print(f"Entry {i}: None\n")

Entry 292:
Text: Looking for a face mask for oily skin $35-$48
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 18, 'B-PRODUCT_TYPE'], [19, 23, 'I-PRODUCT_TYPE'], [24, 27, 'O'], [28, 32, 'B-SKIN_CONCERN'], [33, 37, 'O'], [38, 42, 'B-MIN_PRICE'], [42, 45, 'B-MAX_PRICE']]}


Entry 293:
Text: Looking for a cleanser for acne-prone skin $23-$52
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 22, 'B-PRODUCT_TYPE'], [23, 26, 'O'], [27, 37, 'B-SKIN_CONCERN'], [38, 42, 'O'], [43, 47, 'B-MIN_PRICE'], [47, 50, 'B-MAX_PRICE']]}


Entry 294:
Text: Looking for a face mask for combination skin $49-$67
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 18, 'B-PRODUCT_TYPE'], [19, 23, 'I-PRODUCT_TYPE'], [24, 27, 'O'], [28, 39, 'B-SKIN_CONCERN'], [40, 44, 'O'], [45, 49, 'B-MIN_PRICE'], [49, 52, 'B-MAX_PRICE']]}


Entry 295:
Text: Looking for a face mask for sensitive skin $37-$77 with Ceramides without Phthalates
Annotations: {'en

In [10]:
#Checking structure
print(ner_classes)

['B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'B-SKIN_CONCERN', 'B-PREF_INGREDIENT', 'I-PREF_INGREDIENT', 'B-MIN_PRICE', 'B-MAX_PRICE', 'B-EXCL_INGREDIENT', 'I-EXCL_INGREDIENT', 'O']


In [26]:
def split_annotations(data, train_size=210, test_size=45, val_size=45):
    """
    Splits the annotations data into training, testing, and validation sets.

    :param data: List of annotations.
    :param train_size: Size of the training set.
    :param test_size: Size of the test set.
    :param val_size: Size of the validation set.
    :return: Three lists representing the train, test, and validation sets.
    """
    total_length = len(data)
    all_indices = list(range(total_length))

    # Create evenly spaced indices for test and validation sets
    interval = total_length // (test_size + val_size)
    test_indices = all_indices[0:total_length:interval][:test_size]
    val_indices = all_indices[1:total_length:interval][:val_size]

    # Remaining indices for training set
    train_indices = list(set(all_indices) - set(test_indices) - set(val_indices))
    train_set = [data[i] for i in train_indices[:train_size]]

    # Create test and validation sets
    test_set = [data[i] for i in test_indices]
    val_set = [data[i] for i in val_indices]

    return train_set, test_set, val_set


In [28]:
# Split the 'annotations' data
train_df, test_df, val_df = split_annotations(annotated_df['annotations'])

# Check the sizes of each set
len(train_df), len(test_df), len(val_df)

(210, 45, 45)

# Model Main

Reminder of important names:

sephora_df

test_val_df

train_df

## NER - Use this Section!

In [30]:
# Process each annotated text in ner_annotations to align tokens with their labels
from transformers import AutoTokenizer

# Initialize the tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define the global counter for None values (should end up discarding 3)
none_counter = 0

# Define the process_annotation function
def process_annotation(annotation):
    global none_counter

    if annotation is None:
        none_counter += 1
        return None

    content = annotation[0]
    entities = annotation[1]['entities']
    tokenized_input = tokenizer(content, truncation=True, padding='max_length', max_length=512)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input.input_ids)

    # Initialize labels for each token as 'O'
    aligned_labels = ['O'] * len(tokenized_input['input_ids'])

    # Align labels with BERT tokens
    for start, end, label in entities:
        for i in range(start, end + 1):
            if i < len(tokenized_input['input_ids']):
                prefix = 'B-' if i == start else 'I-'
                aligned_labels[i] = prefix + label

    return tokenized_input['input_ids'], tokenized_input['attention_mask'], aligned_labels


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# Process all annotations, excluding None values entirely
processed_data = [process_annotation(annotation) for annotation in train_df if annotation is not None]

# Print the number of None values discarded
print(f"Number of None values discarded: {none_counter}")

Number of None values discarded: 0


In [32]:
# function to correct this wonky thing that happens with the labels
# some extra garbage is added to the front, this strips that off

#might no longer need this...

def correct_labels(labels):
    corrected_labels = []
    for label_seq in labels:
        corrected_seq = []
        for label in label_seq:
            # Handle the 'O' label correctly
            if label == 'O' or label == 'B-O' or label == 'I-O':
                corrected_label = 'O'
            else:
                parts = label.split('-')
                if len(parts) > 2:
                    # Remove the redundant prefix
                    corrected_label = '-'.join(parts[-2:])
                else:
                    corrected_label = label
            corrected_seq.append(corrected_label)
        corrected_labels.append(corrected_seq)
    return corrected_labels

In [33]:
#double check we've dropped our none's
for item in processed_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)


In [34]:
# Function to convert labels to indices
def convert_labels_to_indices(labels, label_map):
    return [[label_map[label] for label in label_seq] for label_seq in labels]

In [35]:
# Extract labels from processed annotations
extracted_labels = [aligned_labels for _, _, aligned_labels in processed_data if aligned_labels is not None]
print(extracted_labels)

[['B-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'I-B-PREF_INGREDIENT', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MIN_PRICE', 'I-B-MIN_PRICE', 'I-B-MIN_PRICE', 'I-B-MIN_PRICE', 'B-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [36]:
# Correct the labels
corrected_labels = correct_labels(extracted_labels)
print(corrected_labels)

[['B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'O', 'O', 'O', 'O', 'O', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'B-PREF_INGREDIENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MIN_PRICE', 'B-MIN_PRICE', 'B-MIN_PRICE', 'B-MIN_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [37]:
# Create a mapping from string labels to unique integers
label_map = {label: i for i, label in enumerate(ner_classes)}

In [38]:
# Convert the corrected labels to numeric indices
numeric_labels = convert_labels_to_indices(corrected_labels, label_map)

In [39]:
# Convert data to tensors and create a DataLoader
input_ids, attention_mask, _ = zip(*processed_data)
input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)
labels_tensor = torch.tensor(numeric_labels, dtype=torch.long)

dataset = TensorDataset(input_ids_tensor, attention_mask_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32)

## Training with BERT

Here's where the fun stuff happens

In [40]:
from transformers import DistilBertForTokenClassification, AdamW

# Define the number of epochs
epochs = 30  # adjust as needed later

# Define the model for DistilBERT
model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_map),  # Number of distinct labels in dataset
    output_attentions=False,
    output_hidden_states=False
)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# Set training params
optimizer = AdamW(model.parameters(), lr=1e-5)  # can adjust the learning rate as we train...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

Set-up WandB before training

In [43]:
# Initialize a new run
wandb.init(project="DLAIE Project F23")

# Track hyperparameters and run metadata
wandb.config = {
  "learning_rate": 1e-5,
  "epochs": 30,
  "batch_size": 32
}

wandb: Currently logged in as: vcrawfordnelson (dlaief23). Use `wandb login --relogin` to force relogin


In [44]:
#training loop
#model is learning from our data
#iterate over the DataLoader, passing data to model
#update model's weights based on the output

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Log batch loss to W&B
        wandb.log({"batch_loss": loss.item()})

        # Backward pass and update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Calculate the average loss over the epoch
    avg_train_loss = total_loss / len(dataloader)

    # Log average training loss to W&B
    wandb.log({"epoch": epoch, "avg_train_loss": avg_train_loss})

    print(f"Average training loss for epoch {epoch}: {avg_train_loss}")



Average training loss for epoch 0: 1.969543354851859
Average training loss for epoch 1: 1.3242849452154977
Average training loss for epoch 2: 0.6601211513791766
Average training loss for epoch 3: 0.3729364148208073
Average training loss for epoch 4: 0.33765741331236704
Average training loss for epoch 5: 0.3292007829461779
Average training loss for epoch 6: 0.3248066817011152
Average training loss for epoch 7: 0.3182964154652187
Average training loss for epoch 8: 0.313524923154286
Average training loss for epoch 9: 0.30765671815191
Average training loss for epoch 10: 0.3006503369127001
Average training loss for epoch 11: 0.29210098513535093
Average training loss for epoch 12: 0.2808031439781189
Average training loss for epoch 13: 0.2660776994058064
Average training loss for epoch 14: 0.2505626061132976
Average training loss for epoch 15: 0.23855634459427424
Average training loss for epoch 16: 0.22883044609001704
Average training loss for epoch 17: 0.2215819933584758
Average training los

In [45]:
wandb.finish()

avg_train_loss,█▅▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_loss,█▇▅▃▂▂▂▁▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
avg_train_loss,0.18342
batch_loss,0.20802
epoch,29


In [46]:
# Save model for future use when we're happy with performance
model.save_pretrained('./model_save')